# Decoding Logits - Sandbox

In [1]:
%load_ext autoreload
%autoreload 2
import sys
if "../src" not in sys.path:
    sys.path.append("../src")

In [2]:
from datetime import datetime
from pathlib import Path
from itertools import islice
from tqdm.auto import tqdm
import numpy as np
import torch
from torch.utils.data import DataLoader

import datasets
from datasets import Dataset
from transformers import DataCollatorForSeq2Seq
from transformers import MT5TokenizerFast
from vec4gloss import check_hashes
from vec4gloss import Vec4GlossModel

In [3]:
from CwnGraph import CwnImage
cwn = CwnImage.latest()

In [4]:
if torch.cuda.is_available() and "GeForce" not in torch.cuda.get_device_name():
    device = "cuda"
else:
    device = "cpu"
print("Using", device)

Using cuda


In [5]:
torch.manual_seed(12345)
ds_defgen = datasets.load_from_disk("../data/defgen_dataset_cwn")
vec4gloss_model_dir = "../data/models/vec4gloss-defgen-220629-1250"
model = Vec4GlossModel.from_pretrained(vec4gloss_model_dir).to(device)
tokenizer = MT5TokenizerFast.from_pretrained(vec4gloss_model_dir)

## Preprocess functions

In [6]:
max_length = 256

def get_marked_pos(text):
    assert text.count("<") == text.count(">") == 1
    s, e = text.index("<")+1, text.index(">")    
    assert s != e
    return s, e


## Model

In [7]:
def extract_encoder_vector(intext, tokenizer, model):    
    vbatch = tokenizer(intext, return_tensors="pt").to(model.device)
    s,e = get_marked_pos(intext)   
    s = vbatch.char_to_token(s)
    e = vbatch.char_to_token(e)
    vbatch["decoder_start_markers"] = torch.tensor([s]).to(model.device)
    vbatch["decoder_end_markers"] = torch.tensor([e]).to(model.device)
    encoder = model.get_encoder()
    enc_out = encoder(
            input_ids=vbatch["input_ids"], 
            attention_mask=vbatch["attention_mask"])
    enc_vec = enc_out.last_hidden_state[[0],s:e,:] \
                     .mean(1, keepdim=True)
    return enc_vec

def decode_vector(vec, tokenizer, model, max_length=50):
    vgenout = model.generate(decoder_encoder_vector=vec, bos_token_id=0, max_length=max_length)
    return tokenizer.batch_decode(vgenout[:, 1:-1])[0]

def gen_func(tokenizer, model):
    def _gen_func(text):
        enc_vec = extract_encoder_vector(text, tokenizer, model)
        return decode_vector(enc_vec, tokenizer, model)
    return _gen_func
gen = gen_func(tokenizer, model)

## Generation

In [8]:
gen("我line他都不回我，我被<塑膠>了。")

'VC。比喻用特定方式使特定對象受到損害。'

In [9]:
gen("周杰倫今天召<開>演唱會很開心。")

'VC。進行會議。'

In [10]:
gen("我赫然<驚覺>我忘了。")

'VK。形容在意料之外意識到後述事件的存在。'

In [11]:
gen("昨天大家好不容易<團約>上陽明山。")

'D。表共同做。'

In [12]:
gen("我<確認過眼神>，今天的數學考卷很難，完蛋了！")

'VE。經過思考後決定。'

In [13]:
cwn.find_senses(definition="意識到後述事件的存在")

[<CwnSense[07033703](驚，VK): 在意料之外意識到後述事件的存在。>]

## Vector morphing

In [14]:
enc_vec1 = extract_encoder_vector("那是一位嬌豔如<花>的少女。", tokenizer, model)
enc_vec2 = extract_encoder_vector("以動畫方式慢速地顯示字母的每一<筆>一劃。", tokenizer, model)
delta = enc_vec2 - enc_vec1
for i in np.arange(0, 1.01, 0.2):
    print(f"{i:.2f}", decode_vector(enc_vec1+delta*i, tokenizer, model))

0.00 Na。植物的主要器官之一,主要用於繁殖,通常具有顏色鮮豔和形狀漂亮的花瓣。
0.20 Na。植物名,薔薇科櫻屬,多年生草本,葉呈長橢圓形,有花瓣五片,有粉紅、白、紅等顏色。
0.40 Na。植物名,薔薇科櫻屬,多年生草本,葉呈長橢圓形,有花瓣五片,有粉紅、白、紅等顏色。
0.60 Na。書寫筆畫的一種,由左向右上斜的筆畫。
0.80 Na。筆畫的一種,由左向右上斜的筆畫。
1.00 Nf。計算筆畫的單位。


## Vector perturbation

In [16]:
enc_vec = extract_encoder_vector("蘇打綠改名成<魚丁糸>", tokenizer, model)
for _ in range(10):
    rand_vec = torch.randn(768).to(device)
    print(decode_vector(enc_vec+rand_vec*0.04, tokenizer, model))

Na。魚類的通稱,鱗狀,鱗狀,鱗狀,鱗狀,鱗狀,鱗狀,鱗狀,鱗狀,鱗狀,鱗狀,趾尖呈鱗狀,肉質鮮美
Na。魚類的通稱。
Na。魚類的通稱,鱗狀,鱗狀,鱗狀,鱗狀,鱗狀,鱗狀,鱗狀,鱗狀,鱗狀,鱗狀,鱗狀,鱗狀,鱗狀,鱗
Na。用來釣魚的一種技巧,用來輔助釣魚的技巧。
Na。用來魚類的魚類,形狀似魚,體型細長,外型稍微小,外型稍微細,味酸可食,可醃製成多種蜜餞。
Na。常綠或落葉灌木,葉子橢圓形,春夏開花,有粉紅、白、紅等顏色,果實球形,味酸可食,可醃製成多種蜜
Na。落葉喬木,葉卵形,早春開花,花瓣五片,有粉紅、白、紅等顏色,果實球形,味酸可食,可醃製成多種
Na。以魚為形象製成的人造物。
Na。魚類的通稱。
Na。魚類身上用來釣魚的纖維。


In [17]:
torch.manual_seed(12353)
enc_vec = extract_encoder_vector("他這個人很<塑膠>", tokenizer, model)
for scale in range(0, 10, 1):
    rand_vec = torch.randn(768).to(device)
    print(f"{scale/100:.2f}", decode_vector(enc_vec+rand_vec*scale/100, tokenizer, model))

0.00 VH。形容比喻具有不合乎常理的特質。
0.01 VH。形容比喻具有不合乎常理的特質。
0.02 VH。形容比喻個性急躁的。
0.03 VH。形容比喻對特定對象有負面的經驗與感覺。
0.04 VH。形容比喻具有過度執照、不善言辭的。
0.05 VH。形容比喻態度惡劣的。
0.06 VH。形容比喻不具有美的特質。
0.07 VH。形容比喻具有外在特質的。
0.08 VH。形容比喻對思想或立場不專注的。
0.09 VH。形容比喻耗費耗費特定經濟能量的。
